In [ ]:
import json
import csv
import os

def convert_json_to_csv(json_file_path, csv_file_path):
    """
    주어진 JSON 파일에서 데이터를 읽어 지정된 CSV 양식으로 변환하여 저장합니다.

    Args:
        json_file_path (str): 입력 JSON 파일의 경로입니다.
        csv_file_path (str): 출력 CSV 파일의 경로입니다.
    """
    # CSV 헤더 정의
    csv_headers = [
        "요구사항 ID",
        "요구사항유형(기능/비기능)",
        "대분류",
        "중분류",
        "소분류",
        "요구사항 명",
        "요구사항 설명",
        "중요도",
        "난이도"
    ]

    try:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            try:
                requirements_data = json.load(json_file)
            except json.JSONDecodeError as e:
                print(f"오류: JSON 파일 '{json_file_path}' 파싱 중 오류 발생: {e}")
                return
    except FileNotFoundError:
        print(f"오류: JSON 파일 '{json_file_path}'를 찾을 수 없습니다.")
        return
    except Exception as e:
        print(f"오류: JSON 파일 '{json_file_path}'을 여는 중 오류 발생: {e}")
        return

    # JSON 데이터가 리스트 형태인지 확인 (일반적으로 여러 요구사항이 리스트 안에 있음)
    if not isinstance(requirements_data, list):
        # 단일 JSON 객체인 경우, 리스트로 감싸서 처리
        if isinstance(requirements_data, dict):
            requirements_data = [requirements_data]
            print(f"정보: 입력된 JSON 데이터가 단일 객체이므로 리스트로 변환하여 처리합니다.")
        else:
            print(f"오류: JSON 데이터의 최상위 구조가 리스트 또는 단일 객체가 아닙니다 (타입: {type(requirements_data)}).")
            return
            
    try:
        with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as csv_file: # utf-8-sig는 Excel에서 한글 깨짐 방지
            writer = csv.writer(csv_file)
            
            # 1. CSV 헤더 쓰기
            writer.writerow(csv_headers)

            # 2. JSON 데이터 순회하며 CSV 행 쓰기
            for req in requirements_data:
                if not isinstance(req, dict):
                    print(f"경고: 요구사항 데이터 리스트 내에 딕셔너리가 아닌 항목 발견. 건너<0xEB><0x9A><0x84>니다: {req}")
                    continue
                
                # 각 필드의 값을 가져오고, 없는 경우 빈 문자열로 대체 (get 메소드 활용)
                row_to_write = [
                    req.get("id", ""),
                    req.get("type", ""),
                    req.get("category_large", ""),
                    req.get("category_medium", ""),
                    req.get("category_small", ""),
                    req.get("description", ""),  # "요구사항 명"
                    req.get("detailed_description", ""),  # "요구사항 설명"
                    req.get("importance", ""),
                    req.get("difficulty", "")
                ]
                writer.writerow(row_to_write)
        
        print(f"성공: '{json_file_path}' 파일의 내용이 '{csv_file_path}' CSV 파일로 성공적으로 변환되었습니다.")

    except IOError as e:
        print(f"오류: CSV 파일 '{csv_file_path}'을(를) 쓰거나 여는 중 I/O 오류 발생: {e}")
    except Exception as e:
        print(f"CSV 변환 중 예기치 않은 오류 발생: {e}")

if __name__ == '__main__':
    # --- 사용자 설정 ---
    # 실제 입력 JSON 파일 경로와 출력 CSV 파일 경로를 지정해주세요.
    # 예시: input_json_file = 'path/to/your/requirements.json'
    #       output_csv_file = 'path/to/your/output_requirements.csv'

    input_json_file = 'output/enriched_requirements_classified.json' # 여기에 실제 JSON 파일 이름을 넣으세요.
    output_csv_file = 'output/requirements_output.csv' # 여기에 생성될 CSV 파일 이름을 넣으세요.

    # --- 실행 ---
    # 입력 파일이 존재하는지 먼저 확인
    if os.path.exists(input_json_file):
        convert_json_to_csv(input_json_file, output_csv_file)
    else:
        # 테스트를 위한 샘플 JSON 파일 생성 (실제 사용 시 이 부분은 필요 없거나 주석 처리)
        print(f"정보: 테스트를 위해 샘플 JSON 파일 '{input_json_file}'을 생성합니다.")
        sample_data_for_test = [
            {
                "id": "FUNC-001",
                "type": "기능적",
                "description": "사용자 회원가입 기능",
                "acceptance_criteria": "필수 정보 입력 시 회원가입이 성공적으로 완료되어야 한다.",
                "priority": "필수",
                "responsible_module": "회원관리 시스템",
                "source_pages": [5, 6],
                "raw_text_snippet": "사용자는 이메일, 비밀번호, 이름으로 회원가입을 할 수 있다.",
                "detailed_description": "사용자는 서비스 이용을 위해 회원가입을 진행할 수 있다. 이메일 인증 절차가 포함되어야 하며, 비밀번호는 암호화되어 저장되어야 한다.",
                "importance": "상",
                "difficulty": "중",
                "category_large": "사용자관리",
                "category_medium": "계정",
                "category_small": "회원가입"
            },
            { # 사용자께서 제공해주신 예시 데이터
                "id": "FUNC-002",
                "type": "기능적",
                "description": "비대면 서류 작성 시스템을 개발하여 적용한다.",
                "acceptance_criteria": "사용자가 비대면으로 서류를 작성하고 제출할 수 있다.",
                "priority": "필수",
                "responsible_module": "서류 관리 시스템",
                "source_pages": [9, 15],
                "raw_text_snippet": "(O2O)비대면 서류 작성 시스템 개발 적용",
                "detailed_description": "[요구사항]\n비대면 방식으로 서류 작성 및 관리가 가능한 시스템을 개발하여 적용한다.\n\n[대상업무]\n서류 관리 시스템 - 온라인 기능\n\n[요건처리 상세]\n사용자가 시스템에 접속하여 필요한 서류를 선택하고, 입력란에 필요한 정보를 입력한 후 제출하면, 시스템은 입력된 정보를 바탕으로 서류를 자동으로 작성한다. 작성된 서류는 사용자가 확인하고 수정할 수 있으며, 최종적으로 승인하면 서류가 완성된다. 완성된 서류는 시스템 내에서 관리되며, 사용자는 언제든지 접속하여 조회, 수정, 삭제가 가능하다. 또한, 필요한 경우에는 완성된 서류를 다른 사용자와 공유할 수 있다.",
                "importance": "상",
                "difficulty": "중",
                "category_large": "통합고객",
                "category_medium": "서류 관리",
                "category_small": "비대면 서류 작성 및 관리"
            },
            { # 일부 필드가 누락된 데이터 테스트
                "id": "NFR-001",
                "type": "비기능적",
                "description": "시스템 응답 속도 보장",
                # "detailed_description": "모든 주요 기능은 3초 이내에 응답해야 합니다.", # 일부러 누락
                "importance": "높음",
                "difficulty": "상",
                "category_large": "성능",
                "category_small": "응답시간" 
                # "category_medium": "응답성", # 일부러 누락
            }
        ]
        try:
            with open(input_json_file, 'w', encoding='utf-8') as f:
                json.dump(sample_data_for_test, f, ensure_ascii=False, indent=4)
            print(f"샘플 JSON 파일 '{input_json_file}'이 생성되었습니다. 코드를 다시 실행하여 변환을 진행해주세요.")
        except Exception as e:
            print(f"샘플 JSON 파일 생성 중 오류 발생: {e}")